In [ ]:
from google.colab import driv
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 

train_path = "/content/drive/MyDrive/datasets/ta-misogyny/ta_misogyny_olid-train.csv"
dev_path = "/content/drive/MyDrive/datasets/ta-misogyny/ta_misogyny_olid-dev.csv"
test_path = "/content/drive/MyDrive/datasets/ta-misogyny/ta_misogyny_olid-test.csv"


train_df = pd.read_csv(train_path)
dev_df = pd.read_csv(dev_path)
test_df = pd.read_csv(test_path)

In [ ]:
train_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,labels,text
0,0,0,0,Not-Abusive,உங்கள் பேச்சை எதிர்பார்த்தேன் நல்ல விளக்கம் அர...
1,1,1,1,Not-Abusive,எச் ராசாவால் இராமருக்கே இழிவு இவர் எல்லாம் இரா...
2,2,2,2,Not-Abusive,கல்யாணம்னு வரும்போது தான் சாதி முக்கியம் எங்கள...
3,3,3,3,Not-Abusive,Tentu Kottai ஐயா நான் தமிழ் இந்து
4,4,4,4,Abusive,Tamil selvan அடே தேவடியாவுக்கு பிரந்தவனே பச்சத...
...,...,...,...,...,...
2233,2233,2233,2235,Not-Abusive,ராஜா ஏண்டா பின்னாடி பின்னாடி பார்த்து பேசுற மு...
2234,2234,2234,2236,Not-Abusive,பழய சித்தாகதை கிளிக வேண்டும் ஆண்ணா
2235,2235,2235,2237,Not-Abusive,SRI NARAYANA நீ ஒரு ஆரிய இந்து சரியா
2236,2236,2236,2238,Not-Abusive,அறியா வயதில் குழந்தைகளை அடித்து சொல்லிக்கொடுத்...


In [ ]:
print(train_df.shape)
print(dev_df.shape)
print(test_df.shape)

(2238, 5)
(560, 5)
(700, 5)


In [ ]:
#convert labels for Not Offensive
train_df['labels'] = train_df['labels'].replace(['Not-Offensive'],['Not-Abusive'])
dev_df['labels'] = dev_df['labels'].replace(['Not-Offensive'],['Not-Abusive'])
test_df['labels'] = test_df['labels'].replace(['Not-Offensive'],['Not-Abusive'])

train_df['labels'] = train_df['labels'].replace(['Offensive'],['Abusive'])
dev_df['labels'] = dev_df['labels'].replace(['Offensive'],['Abusive'])
test_df['labels'] = test_df['labels'].replace(['Offensive'],['Abusive'])

In [ ]:
train_df.to_csv(train_path)
dev_df.to_csv(dev_path)
test_df.to_csv(test_path)

In [ ]:
cross_val_df = pd.concat([train_df,dev_df,test_df],axis=0)
cross_val_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,labels,text
0,0,0,0,Not-Abusive,உங்கள் பேச்சை எதிர்பார்த்தேன் நல்ல விளக்கம் அர...
1,1,1,1,Not-Abusive,எச் ராசாவால் இராமருக்கே இழிவு இவர் எல்லாம் இரா...
2,2,2,2,Not-Abusive,கல்யாணம்னு வரும்போது தான் சாதி முக்கியம் எங்கள...
3,3,3,3,Not-Abusive,Tentu Kottai ஐயா நான் தமிழ் இந்து
4,4,4,4,Abusive,Tamil selvan அடே தேவடியாவுக்கு பிரந்தவனே பச்சத...
...,...,...,...,...,...
695,695,695,695,Not-Abusive,சினிவாசன் அய்யாக்கு நன்றி...
696,696,696,696,Not-Abusive,ஊருக்கே தெரியும் அவர் பிராமணன் என்று.அவர் ஒரு தலை
697,697,697,697,Not-Abusive,ஐயா அவர்கள் சரியான விளக்கம் பார்பனர்கள் வேறு ப...
698,698,698,698,Abusive,Nandha Kumar அட நாயே; ; வந்தேறி பிராமண நாய...


In [ ]:
cross_val_df['labels'].value_counts()

Not-Abusive    2414
Abusive        1084
Name: labels, dtype: int64

In [ ]:
cross_val_df = cross_val_df[cross_val_df.labels != 'Not-Tamil']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression


from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
import pandas
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
cross_val_df = cross_val_df.dropna()

#Linear SVC 

In [ ]:
SVC__tfidf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', LinearSVC()),
                    ])


SVC__bow = Pipeline([('vect', CountVectorizer()),
                    ('clf', LinearSVC()),
                    ])

## SVC TFIDF

### Macro F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred)) # print classification report
    return f1_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(SVC__tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score) 

              precision    recall  f1-score   support

     Abusive       0.74      0.63      0.68       217
 Not-Abusive       0.84      0.90      0.87       483

    accuracy                           0.82       700
   macro avg       0.79      0.76      0.78       700
weighted avg       0.81      0.82      0.81       700

              precision    recall  f1-score   support

     Abusive       0.67      0.55      0.60       217
 Not-Abusive       0.81      0.88      0.84       483

    accuracy                           0.78       700
   macro avg       0.74      0.71      0.72       700
weighted avg       0.77      0.78      0.77       700

              precision    recall  f1-score   support

     Abusive       0.67      0.56      0.61       216
 Not-Abusive       0.82      0.88      0.85       483

    accuracy                           0.78       699
   macro avg       0.74      0.72      0.73       699
weighted avg       0.77      0.78      0.77       699

              preci

In [ ]:

nested_score.mean()

0.7346854564578191

In [ ]:
nested_score.std()

0.02133533896647216

### Weighted F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred)) # print classification report
    return f1_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(SVC__tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score) 

              precision    recall  f1-score   support

     Abusive       0.74      0.63      0.68       217
 Not-Abusive       0.84      0.90      0.87       483

    accuracy                           0.82       700
   macro avg       0.79      0.76      0.78       700
weighted avg       0.81      0.82      0.81       700

              precision    recall  f1-score   support

     Abusive       0.67      0.55      0.60       217
 Not-Abusive       0.81      0.88      0.84       483

    accuracy                           0.78       700
   macro avg       0.74      0.71      0.72       700
weighted avg       0.77      0.78      0.77       700

              precision    recall  f1-score   support

     Abusive       0.67      0.56      0.61       216
 Not-Abusive       0.82      0.88      0.85       483

    accuracy                           0.78       699
   macro avg       0.74      0.72      0.73       699
weighted avg       0.77      0.78      0.77       699

              preci

In [ ]:
nested_score.mean()

0.7788148083211915

In [ ]:
nested_score.std()

0.017330719032445974

### Macro Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    print(classification_report(y_true, y_pred)) # print classification report
    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(SVC__tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score) 

              precision    recall  f1-score   support

     Abusive       0.74      0.63      0.68       217
 Not-Abusive       0.84      0.90      0.87       483

    accuracy                           0.82       700
   macro avg       0.79      0.76      0.78       700
weighted avg       0.81      0.82      0.81       700

              precision    recall  f1-score   support

     Abusive       0.67      0.55      0.60       217
 Not-Abusive       0.81      0.88      0.84       483

    accuracy                           0.78       700
   macro avg       0.74      0.71      0.72       700
weighted avg       0.77      0.78      0.77       700

              precision    recall  f1-score   support

     Abusive       0.67      0.56      0.61       216
 Not-Abusive       0.82      0.88      0.85       483

    accuracy                           0.78       699
   macro avg       0.74      0.72      0.73       699
weighted avg       0.77      0.78      0.77       699

              preci

In [ ]:
precision_score.mean()

0.7534175913398048

In [ ]:
precision_score.std()

0.020376897375067753

### Weighted Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    print(classification_report(y_true, y_pred)) # print classification report
    return precision_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(SVC__tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score) 

              precision    recall  f1-score   support

     Abusive       0.74      0.63      0.68       217
 Not-Abusive       0.84      0.90      0.87       483

    accuracy                           0.82       700
   macro avg       0.79      0.76      0.78       700
weighted avg       0.81      0.82      0.81       700

              precision    recall  f1-score   support

     Abusive       0.67      0.55      0.60       217
 Not-Abusive       0.81      0.88      0.84       483

    accuracy                           0.78       700
   macro avg       0.74      0.71      0.72       700
weighted avg       0.77      0.78      0.77       700

              precision    recall  f1-score   support

     Abusive       0.67      0.56      0.61       216
 Not-Abusive       0.82      0.88      0.85       483

    accuracy                           0.78       699
   macro avg       0.74      0.72      0.73       699
weighted avg       0.77      0.78      0.77       699

              preci

In [ ]:
precision_score.mean()

0.7780869169545792

In [ ]:
precision_score.std()

0.017603268448311983

### Macro Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    print(classification_report(y_true, y_pred)) # print classification report
    return recall_score(y_true, y_pred,average="macro")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(SVC__tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score) 

              precision    recall  f1-score   support

     Abusive       0.74      0.63      0.68       217
 Not-Abusive       0.84      0.90      0.87       483

    accuracy                           0.82       700
   macro avg       0.79      0.76      0.78       700
weighted avg       0.81      0.82      0.81       700

              precision    recall  f1-score   support

     Abusive       0.67      0.55      0.60       217
 Not-Abusive       0.81      0.88      0.84       483

    accuracy                           0.78       700
   macro avg       0.74      0.71      0.72       700
weighted avg       0.77      0.78      0.77       700

              precision    recall  f1-score   support

     Abusive       0.67      0.56      0.61       216
 Not-Abusive       0.82      0.88      0.85       483

    accuracy                           0.78       699
   macro avg       0.74      0.72      0.73       699
weighted avg       0.77      0.78      0.77       699

              preci

In [ ]:
recall_score.mean()

0.7239181102286559

In [ ]:
recall_score.std()

0.02135660834769216

### Weighted Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    print(classification_report(y_true, y_pred)) # print classification report
    return recall_score(y_true, y_pred,average="weighted")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(SVC__tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score) 

              precision    recall  f1-score   support

     Abusive       0.74      0.63      0.68       217
 Not-Abusive       0.84      0.90      0.87       483

    accuracy                           0.82       700
   macro avg       0.79      0.76      0.78       700
weighted avg       0.81      0.82      0.81       700

              precision    recall  f1-score   support

     Abusive       0.67      0.55      0.60       217
 Not-Abusive       0.81      0.88      0.84       483

    accuracy                           0.78       700
   macro avg       0.74      0.71      0.72       700
weighted avg       0.77      0.78      0.77       700

              precision    recall  f1-score   support

     Abusive       0.67      0.56      0.61       216
 Not-Abusive       0.82      0.88      0.85       483

    accuracy                           0.78       699
   macro avg       0.74      0.72      0.73       699
weighted avg       0.77      0.78      0.77       699

              preci

In [ ]:
recall_score.mean()

0.7855242182709994

In [ ]:
recall_score.std()

0.01622491920091351

### Accuracy

In [ ]:
nested_score = cross_val_score(SVC__tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring="accuracy")

In [ ]:
nested_score.mean()

0.7855242182709994

In [ ]:
nested_score.std()

0.01622491920091351

## SVC BOW

## Macro Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    print(classification_report(y_true, y_pred)) # print classification report
    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(SVC__tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score) 

              precision    recall  f1-score   support

     Abusive       0.74      0.63      0.68       217
 Not-Abusive       0.84      0.90      0.87       483

    accuracy                           0.82       700
   macro avg       0.79      0.76      0.78       700
weighted avg       0.81      0.82      0.81       700

              precision    recall  f1-score   support

     Abusive       0.67      0.55      0.60       217
 Not-Abusive       0.81      0.88      0.84       483

    accuracy                           0.78       700
   macro avg       0.74      0.71      0.72       700
weighted avg       0.77      0.78      0.77       700

              precision    recall  f1-score   support

     Abusive       0.67      0.56      0.61       216
 Not-Abusive       0.82      0.88      0.85       483

    accuracy                           0.78       699
   macro avg       0.74      0.72      0.73       699
weighted avg       0.77      0.78      0.77       699

              preci

In [ ]:
precision_score.mean()

0.7534175913398048

In [ ]:
precision_score.std()

0.020376897375067753

### Macro F1

In [ ]:
SVC

NameError: ignored

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred)) # print classification report
    return f1_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(SVC__bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score) 

In [ ]:
nested_score.mean()

0.7855242182709994

In [ ]:
nested_score.std()

0.01622491920091351

### Confusion Matrix

In [ ]:
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(SVC__tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold)
conf_mat = confusion_matrix(cross_val_df['labels'].values, y_pred)

print(conf_mat)

[[ 609  475]
 [ 275 2138]]


### Weighted F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred)) # print classification report
    return f1_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(SVC__bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score) 

              precision    recall  f1-score   support

     Abusive       0.68      0.65      0.66       217
 Not-Abusive       0.84      0.86      0.85       483

    accuracy                           0.80       700
   macro avg       0.76      0.75      0.76       700
weighted avg       0.79      0.80      0.79       700

              precision    recall  f1-score   support

     Abusive       0.65      0.57      0.60       217
 Not-Abusive       0.82      0.86      0.84       483

    accuracy                           0.77       700
   macro avg       0.73      0.71      0.72       700
weighted avg       0.76      0.77      0.77       700

              precision    recall  f1-score   support

     Abusive       0.63      0.55      0.59       216
 Not-Abusive       0.81      0.86      0.83       483

    accuracy                           0.76       699
   macro avg       0.72      0.70      0.71       699
weighted avg       0.76      0.76      0.76       699

              preci

In [ ]:
nested_score.mean()

0.7657985730131502

In [ ]:
nested_score.std()

0.014676404848267176

### Confusion Matrix

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(SVC__bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold)
conf_mat = confusion_matrix(cross_val_df['labels'].values, y_pred)

In [ ]:
conf_mat

array([[ 614,  470],
       [ 335, 2078]])

#### Macro Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    print(classification_report(y_true, y_pred)) # print classification report
    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(SVC__bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score) 

              precision    recall  f1-score   support

     Abusive       0.68      0.65      0.66       217
 Not-Abusive       0.84      0.86      0.85       483

    accuracy                           0.79       700
   macro avg       0.76      0.75      0.76       700
weighted avg       0.79      0.79      0.79       700

              precision    recall  f1-score   support

     Abusive       0.65      0.57      0.60       217
 Not-Abusive       0.82      0.86      0.84       483

    accuracy                           0.77       700
   macro avg       0.73      0.71      0.72       700
weighted avg       0.76      0.77      0.77       700

              precision    recall  f1-score   support

     Abusive       0.63      0.55      0.59       216
 Not-Abusive       0.81      0.86      0.83       483

    accuracy                           0.76       699
   macro avg       0.72      0.70      0.71       699
weighted avg       0.75      0.76      0.76       699

              preci

In [ ]:
print(precision_score.mean())

0.7310120861527588


In [ ]:
print(precision_score.std())

0.015098351753092773


### Weighted Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return precision_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(SVC__bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.7642499722089258
0.014891340372114894


### Macro Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    print(classification_report(y_true, y_pred)) # print classification report
    return recall_score(y_true, y_pred,average="macro")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(SVC__bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score) 

              precision    recall  f1-score   support

     Abusive       0.68      0.65      0.66       217
 Not-Abusive       0.84      0.86      0.85       483

    accuracy                           0.80       700
   macro avg       0.76      0.75      0.76       700
weighted avg       0.79      0.80      0.79       700

              precision    recall  f1-score   support

     Abusive       0.65      0.57      0.60       217
 Not-Abusive       0.82      0.86      0.84       483

    accuracy                           0.77       700
   macro avg       0.73      0.71      0.72       700
weighted avg       0.76      0.77      0.77       700

              precision    recall  f1-score   support

     Abusive       0.63      0.55      0.59       216
 Not-Abusive       0.81      0.86      0.83       483

    accuracy                           0.76       699
   macro avg       0.72      0.70      0.71       699
weighted avg       0.76      0.76      0.76       699

              preci

In [ ]:
recall_score.mean()

0.7147095434914859

In [ ]:
recall_score.std()

0.020470133792572443


### Weighted Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):
    return recall_score(y_true, y_pred,average="weighted")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(SVC__bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score.mean())
print(recall_score.std()) 

0.7703674637236869
0.012510495959933038


### Accuracy

In [ ]:
nested_score = cross_val_score(SVC__bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring="accuracy")
nested_score.mean()

0.7709388922951155

In [ ]:
nested_score.std()

0.01308725426097974

# Random Forest

In [ ]:

rf_tfidf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', RandomForestClassifier()),
                    ])


rf_bow = Pipeline([('vect', CountVectorizer()),
                    ('clf', RandomForestClassifier()),
                    ])

## Random Forest TFIDF

### Macro F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return f1_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(rf_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean()) 
print(nested_score.std())

0.7316971535753638
0.022797221925954222


### Weighted F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return f1_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(rf_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean()) 
print(nested_score.std())

0.7791113438083012
0.022464944773362912


### Macro Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(rf_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.7753252725053834
0.028756724704001282


### Weighted Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return precision_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(rf_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.7835536263396788
0.02418981415306221


### Macro Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):
    return recall_score(y_true, y_pred,average="macro")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(rf_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score.mean()) 
print(recall_score.std()) 

0.7141850478394536
0.023992985916912426


### Weighted Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):
    return recall_score(y_true, y_pred,average="weighted")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(rf_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score.mean()) 
print(recall_score.std()) 

0.7895319844676069
0.01989134407090116


### Accuracy

In [ ]:
nested_score = cross_val_score(rf_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring="accuracy")

In [ ]:
nested_score.mean()

0.7895319844676068

In [ ]:
nested_score.std()

0.019854383186432213

### Confusion Matrix

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(rf_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold)
conf_mat = confusion_matrix(cross_val_df['labels'].values, y_pred)

In [ ]:
conf_mat

array([[ 544,  540],
       [ 191, 2222]])

## Random Forest with BOW

### Macro F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return f1_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(rf_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean()) 
print(nested_score.std())

0.735226506442555
0.020019620788667765


### Weighted F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return f1_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(rf_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean()) 
print(nested_score.std())

0.780449250865904
0.016045388465381085


### Macro Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(rf_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.7666331279797436
0.01669885822704232


### Weighted Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return precision_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(rf_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.7812833816059352
0.01813529775282654


### Macro Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):
    return recall_score(y_true, y_pred,average="macro")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(rf_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score.mean())
print(recall_score.std()) 

0.7175893194502833
0.020610521184346316


### Weighted Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):
    return recall_score(y_true, y_pred,average="weighted")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(rf_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score.mean())
print(recall_score.std()) 

0.7903919885550786
0.012833278286177637


### Confusion Matrix

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(rf_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold)
conf_mat = confusion_matrix(cross_val_df['labels'].values, y_pred)

In [ ]:
conf_mat

array([[ 583,  501],
       [ 224, 2189]])

## MultinomialNB

In [ ]:
mnb_tfidf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultinomialNB()),
                    ])


mnb_bow = Pipeline([('vect', CountVectorizer()),
                    ('clf', MultinomialNB()),
                    ])

## MNB TFIDF

### Macro F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return f1_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(mnb_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean()) 
print(nested_score.std())

0.6810023452379117
0.01413701812908041


### Weighted F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return f1_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(mnb_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean()) 
print(nested_score.std())

0.7477809437060791
0.010974925404249974


### Macro Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(mnb_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.7932824439939393
0.01570090533166209


### Weighted Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return precision_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(mnb_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.7851057258628511
0.012054586236065892


### Macro Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    return recall_score(y_true, y_pred,average="macro")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(mnb_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score.mean())
print(recall_score.std()) 

0.6644769533215729
0.011581020194911142


### Weighted Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    return recall_score(y_true, y_pred,average="weighted")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(mnb_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score.mean())
print(recall_score.std()) 

0.7778099325567137
0.008967228714495155


### Confusion Matrx|

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(mnb_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold)
conf_mat = confusion_matrix(cross_val_df['labels'].values, y_pred)

In [ ]:
conf_mat

array([[ 397,  687],
       [  90, 2323]])

## MNB BOW

### Macro F1 Score

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return f1_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(mnb_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean()) 
print(nested_score.std())

0.7411141300257312
0.01789219418804216


### Weighted F1 Score

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return f1_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(mnb_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean()) 
print(nested_score.std())

0.782759662653062
0.015002852169099362


### Macro Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(mnb_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.75447723698343
0.018862295363135884


### Weighted Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return precision_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(mnb_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.7815613787842347
0.015330948652514425


### Macro Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    return recall_score(y_true, y_pred,average="macro")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(mnb_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score.mean())
print(recall_score.std()) 

0.7324893033216668
0.017595371415746228


### Weighted Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    return recall_score(y_true, y_pred,average="weighted")


kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(mnb_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score.mean())
print(recall_score.std()) 

0.7875315757204169
0.01481635654492127


### Confusion Matrix

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(mnb_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold)
conf_mat = confusion_matrix(cross_val_df['labels'].values, y_pred)
print(conf_mat)

[[ 637  447]
 [ 296 2117]]


# Decision Tree

In [ ]:
dt_tfidf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', DecisionTreeClassifier()),
                    ])


dt_bow = Pipeline([('vect', CountVectorizer()),
                    ('clf',   DecisionTreeClassifier()),
                    ])

## DT TFIDF

### Macro F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return f1_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(dt_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean()) 
print(nested_score.std())

0.6948262368285473
0.012556131066295238


### Weighted F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return f1_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(dt_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean()) 
print(nested_score.std())

0.7417232530374308
0.011836022239963603


### Macro Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(dt_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.693276137715771
0.009395125335267715


### Weighted Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return precision_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(mnb_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.7815613787842347
0.015330948652514425


### Macro Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    return recall_score(y_true, y_pred,average="macro")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(dt_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score.mean())
print(recall_score.std()) 

0.6967521163882602
0.01635528896118793


### Weighted Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    return recall_score(y_true, y_pred,average="weighted")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(dt_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score.mean())
print(recall_score.std()) 

0.7394870222767218
0.014633297481573528


### Confusion Matrix

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(dt_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold)
conf_mat = confusion_matrix(cross_val_df['labels'].values, y_pred)
print(conf_mat)

[[ 616  468]
 [ 436 1977]]


## DT BOW

### Macro F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return f1_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(dt_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean()) 
print(nested_score.std())

0.6846552263933979
0.017954940071888713


### Weighted F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return f1_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(dt_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean()) 
print(nested_score.std())

0.7418769672615223
0.014080470858541269


### Macro Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(dt_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.6856222949504873
0.016688544234197276


### Weighted Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return precision_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(dt_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.7388184937187011
0.012203111345149181


### Macro Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    return recall_score(y_true, y_pred,average="macro")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(dt_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score.mean())
print(recall_score.std()) 

0.691060900938733
0.01421159444425028


### Weighted Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    return recall_score(y_true, y_pred,average="weighted")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(dt_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score.mean())
print(recall_score.std()) 

0.732056815859391
0.020907165904080722


### Confusion Matrix

In [ ]:

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(dt_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold)
conf_mat = confusion_matrix(cross_val_df['labels'].values, y_pred)
print(conf_mat)

[[ 637  447]
 [ 489 1924]]


# Logitic Regression

In [ ]:
lr_tfidf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', LogisticRegression()),
                    ])


lr_bow = Pipeline([('vect', CountVectorizer()),
                    ('clf',   LogisticRegression()),
                    ])

## LR TFIDF

### Macro F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return f1_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(lr_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean()) 
print(nested_score.std())

0.7244140375457991
0.024140302346924367


### Weighted F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return f1_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(lr_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean()) 
print(nested_score.std())

0.7768468962216782
0.01832765813899637


### Macro Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(lr_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.784689202462856
0.017311964395202654


### Weighted Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return precision_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(lr_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.7900833659354463
0.01597271112003094


### Macro Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    return recall_score(y_true, y_pred,average="macro")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(lr_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score.mean())
print(recall_score.std()) 

0.7055971849682894
0.02287750816467066


### Weighted Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    return recall_score(y_true, y_pred,average="weighted")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(lr_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score.mean())
print(recall_score.std()) 

0.7935360719395055
0.01482509803447973


### Confusion Matrix

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(lr_tfidf, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold)
conf_mat = confusion_matrix(cross_val_df['labels'].values, y_pred)
print(conf_mat)

[[ 514  570]
 [ 152 2261]]


## LR BOW

### Macro F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return f1_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(lr_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean()) 
print(nested_score.std())

0.7369184364293623
0.015954261169327008


### Weighted F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return f1_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
nested_score = cross_val_score(lr_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score.mean()) 
print(nested_score.std())

0.7827044833708066
0.012528198912697375


### Macro Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(lr_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.7667967655062726
0.014789921729716774


### Weighted Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):
    return precision_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(lr_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score.mean())
print(precision_score.std()) 

0.7848078730524277
0.012605097113859248


### Macro Recall|

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    #print(classification_report(y_true, y_pred)) # print classification report
    return recall_score(y_true, y_pred,average="macro")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(lr_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score) 
print("Mean of recall",recall_score.mean())
print("STD of recall",recall_score.std())


[0.74190209 0.69478394 0.72065505 0.73269977 0.72290476]
Mean of recall 0.7225891224613095
STD of recall 0.015825196130110054


### Weighted Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    #print(classification_report(y_true, y_pred)) # print classification report
    return recall_score(y_true, y_pred,average="weighted")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(lr_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score) 
print("Mean of recall",recall_score.mean())
print("STD of recall",recall_score.std())


[0.80142857 0.77142857 0.7925608  0.80257511 0.7925608 ]
Mean of recall 0.7921107704884529
STD of recall 0.011175707336179157


### Confuion Matrix

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(lr_bow, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold)
conf_mat = confusion_matrix(cross_val_df['labels'].values, y_pred)
print(conf_mat)

[[ 585  499]
 [ 228 2185]]


# FASTTEXT

In [ ]:
!pip install fasttext -q

     |████████████████████████████████| 68 kB 3.9 MB/s 


In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ta.300.bin.gz

--2022-01-29 12:39:58--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ta.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4510026494 (4.2G) [application/octet-stream]
Saving to: ‘cc.ta.300.bin.gz’

cc.ta.300.bin.gz    100%[===================>]   4.20G  42.5MB/s    in 97s     

2022-01-29 12:41:35 (44.6 MB/s) - ‘cc.ta.300.bin.gz’ saved [4510026494/4510026494]



In [ ]:
!gzip -d /content/cc.ta.300.bin.gz

In [ ]:
import fasttext

In [ ]:
T_ft = fasttext.load_model('/content/cc.ta.300.bin')

#T_ft.get_dimension()
#fasttext.util.reduce_model(T_ft, 100)

In [ ]:
from sklearn.naive_bayes import GaussianNB

dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
lr = LogisticRegression()
svc = LinearSVC()
nb = GaussianNB()

In [ ]:
x = cross_val_df['text'].values
y = cross_val_df['labels'].values

x = [T_ft.get_word_vector(words)for words in x]
#y = [T_ft.get_word_vector(words)for words in y]
  

## LR with Fasttext

### Macro F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return f1_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)

nested_score = cross_val_score(lr, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score) 
print(f"Mean of F1 {nested_score.mean()}")
print(f"STD of F1 {nested_score.std()}")

[0.40828402 0.40728196 0.41358374 0.4081287  0.40712468]
Mean of F1 0.40888062279930476
STD of F1 0.002394980374248761


### Weighted F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return f1_score(y_true, y_pred,average="weighted")

nested_score = cross_val_score(lr, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score) 
print(f"Mean of F1 {nested_score.mean()}")
print(f"STD of F1 {nested_score.std()}")

[0.56343195 0.56204911 0.56804158 0.56285561 0.56147095]
Mean of F1 0.5635698399974609
STD of F1 0.002334246621323106


### Macro Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(lr, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score) 
print(precision_score.mean())
print(precision_score.std())

[0.345      0.34455587 0.84598854 0.34477825 0.34433286]
0.4449311044700412
0.2005288404140211


### Weighted Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return precision_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(lr, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score) 
print(precision_score.mean())
print(precision_score.std())

[0.4761     0.47548711 0.78716013 0.47548818 0.47487392]
0.537821867028554
0.12466973259610988


### Macro Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    #print(classification_report(y_true, y_pred)) # print classification report
    return recall_score(y_true, y_pred,average="macro")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(lr, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score) 
print("Mean of recall",recall_score.mean())
print("STD of recall",recall_score.std())


[0.5        0.49792961 0.50231481 0.5        0.49792531]
Mean of recall 0.4996339465286786
STD of recall 0.0016296781422598457


### Weighted Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    #print(classification_report(y_true, y_pred)) # print classification report
    return recall_score(y_true, y_pred,average="weighted")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(lr, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score) 
print("Mean of recall",recall_score.mean())
print("STD of recall",recall_score.std())


[0.69       0.68714286 0.69241774 0.68955651 0.68669528]
Mean of recall 0.6891624770079706
STD of recall 0.0020793927715807253


In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(lr, X=x, y=y, cv=kfold)
conf_mat = confusion_matrix(cross_val_df['labels'].values, y_pred)
print(conf_mat)

[[   1 1083]
 [   4 2409]]


### Confusion mATRIX

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(lr, X=x, y=y, cv=kfold)
conf_mat = confusion_matrix(y, y_pred)
print(conf_mat)

[[   1 1083]
 [   4 2409]]


## DT with Fasttext

### Macro F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return f1_score(y_true, y_pred,average="macro")


nested_score = cross_val_score(dt, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score) 
print(f"Mean of F1 {nested_score.mean()}")
print(f"STD of F1 {nested_score.std()}")

[0.61842332 0.60207666 0.61182614 0.61078057 0.60498174]
Mean of F1 0.6096176863383679
STD of F1 0.005694346802011977


### Weighted F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return f1_score(y_true, y_pred,average="weighted")


nested_score = cross_val_score(dt, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score) 
print(f"Mean of F1 {nested_score.mean()}")
print(f"STD of F1 {nested_score.std()}")

[0.65711096 0.66354206 0.64880452 0.67177548 0.66662423]
Mean of F1 0.6615714517526528
STD of F1 0.007955208682544068


### Macro Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(dt, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score) 
print(precision_score.mean())
print(precision_score.std())

[0.61482205 0.62275273 0.60703614 0.61908607 0.61437141]
0.6156136790125262
0.0052681597345435325


### Weighted Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(dt, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score) 
print(precision_score.mean())
print(precision_score.std())

[0.61902298 0.61407346 0.62031826 0.62174402 0.60685027]
0.6164017994588121
0.005429574781413764


### Macro Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    #print(classification_report(y_true, y_pred)) # print classification report
    return recall_score(y_true, y_pred,average="macro")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(dt, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score) 
print("Mean of recall",recall_score.mean())
print("STD of recall",recall_score.std())


[0.63524344 0.61714419 0.59763918 0.62644129 0.61780312]
Mean of recall 0.618854243847624
STD of recall 0.012492214616893635


### Weighted Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    #print(classification_report(y_true, y_pred)) # print classification report
    return recall_score(y_true, y_pred,average="weighted")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(dt, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score) 
print("Mean of recall",recall_score.mean())
print("STD of recall",recall_score.std())


[0.66       0.65714286 0.64663805 0.66666667 0.68240343]
Mean of recall 0.662570202329859
STD of recall 0.011832442118141473


### cONFUSION Matrix

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(dt, X=x, y=y, cv=kfold)
conf_mat = confusion_matrix(y, y_pred)
print(conf_mat)

[[ 534  550]
 [ 640 1773]]


## RF with Fasttext

### Macro F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return f1_score(y_true, y_pred,average="macro")


nested_score = cross_val_score(rf, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score) 
print(f"Mean of F1 {nested_score.mean()}")
print(f"STD of F1 {nested_score.std()}")

[0.73038799 0.68275277 0.67011388 0.70270363 0.68931106]
Mean of F1 0.6950538669758928
STD of F1 0.020558789613369727


### Weighted F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return f1_score(y_true, y_pred,average="weighted")


nested_score = cross_val_score(rf, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score) 
print(f"Mean of F1 {nested_score.mean()}")
print(f"STD of F1 {nested_score.std()}")

[0.8062242  0.73713963 0.74323951 0.74206312 0.75818025]
Mean of F1 0.7573693420516523
STD of F1 0.02541900081235339


### Macro Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(rf, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score) 
print(precision_score.mean())
print(precision_score.std())

[0.84772272 0.77829879 0.79182214 0.78149111 0.76352061]
0.7925710741945389
0.029026396889160873


### Weighted Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return precision_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(rf, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score) 
print(precision_score.mean())
print(precision_score.std())

[0.83075182 0.77243499 0.77979776 0.77645461 0.78494488]
0.7888768127761611
0.021335650283382582


### Macro Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    #print(classification_report(y_true, y_pred)) # print classification report
    return recall_score(y_true, y_pred,average="macro")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(rf, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score) 
print("Mean of recall",recall_score.mean())
print("STD of recall",recall_score.std())


[0.70717291 0.65544647 0.6531516  0.67786871 0.65920607]
Mean of recall 0.670569152952339
STD of recall 0.020269706376222867


### Weighted Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    #print(classification_report(y_true, y_pred)) # print classification report
    return recall_score(y_true, y_pred,average="weighted")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(rf, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score) 
print("Mean of recall",recall_score.mean())
print("STD of recall",recall_score.std())


[0.80571429 0.77714286 0.78397711 0.77825465 0.77110157]
Mean of recall 0.7832380952380953
STD of recall 0.011958320081456987


### Confusion Matrix

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(rf, X=x, y=y, cv=kfold)
conf_mat = confusion_matrix(y, y_pred)
print(conf_mat)

[[ 414  670]
 [ 107 2306]]


## SVC with Fasttext

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return f1_score(y_true, y_pred,average="macro")

nested_score = cross_val_score(svc,X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score) 
print(f"Mean of F1 {nested_score.mean()}")
print(f"STD of F1 {nested_score.std()}")

[0.62719455 0.60479072 0.60790695 0.59475146 0.62835718]
Mean of F1 0.6126001732252797
STD of F1 0.013136738841594785


In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return f1_score(y_true, y_pred,average="weighted")

nested_score = cross_val_score(svc,X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score) 
print(f"Mean of F1 {nested_score.mean()}")
print(f"STD of F1 {nested_score.std()}")

[0.71037277 0.69446935 0.69704013 0.68706433 0.70978984]
Mean of F1 0.6997472861822878
STD of F1 0.009053137162568309


### Macro Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(svc, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score) 
print(precision_score.mean())
print(precision_score.std())

[0.78392604 0.77007874 0.76656669 0.76329316 0.76542178]
0.7698572821277427
0.007369783128477504


### Wighted Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return precision_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(svc, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score) 
print(precision_score.mean())
print(precision_score.std())

[0.77042905 0.75870866 0.75721669 0.75304595 0.75911731]
0.7597035347736094
0.005777160501896403


###  Macro Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    #print(classification_report(y_true, y_pred)) # print classification report
    return recall_score(y_true, y_pred,average="macro")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(svc, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score) 
print("Mean of recall",recall_score.mean())
print("STD of recall",recall_score.std())


[0.62248714 0.60635811 0.60819243 0.59941775 0.62291814]
Mean of recall 0.6118747151203809
STD of recall 0.009313957567854443


### Weighted Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    #print(classification_report(y_true, y_pred)) # print classification report
    return recall_score(y_true, y_pred,average="weighted")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(svc, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score) 
print("Mean of recall",recall_score.mean())
print("STD of recall",recall_score.std())


[0.75571429 0.74571429 0.74678112 0.74105866 0.75250358]
Mean of recall 0.7483543838136113
STD of recall 0.005177970681698347


## Confusion Matrix

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(svc, X=x, y=y, cv=kfold)
conf_mat = confusion_matrix(y, y_pred)
print(conf_mat)

[[ 274  810]
 [  70 2343]]


## Naive bayes with fasttext

### Macro F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return f1_score(y_true, y_pred,average="macro")

nested_score = cross_val_score(nb,X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score) 
print(f"Mean of F1 {nested_score.mean()}")
print(f"STD of F1 {nested_score.std()}")

[0.4628071  0.43178072 0.42465813 0.42829047 0.46053382]
Mean of F1 0.44161404711349794
STD of F1 0.01654580411600696


### Weighted F1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return f1_score(y_true, y_pred,average="weighted")

nested_score = cross_val_score(nb,X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(nested_score) 
print(f"Mean of F1 {nested_score.mean()}")
print(f"STD of F1 {nested_score.std()}")

[0.43918596 0.41080531 0.40516531 0.40038371 0.44531927]
Mean of F1 0.42017191186606573
STD of F1 0.01843055940944883


### Macro Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return precision_score(y_true, y_pred,average="macro")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(nb, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score) 
print(precision_score.mean())
print(precision_score.std())

[0.63761994 0.56902033 0.55241868 0.59827377 0.58815117]
0.5890967802164712
0.028941165579279778


### Weighted Precision

In [ ]:
from sklearn.metrics import classification_report, accuracy_score,precision_score, make_scorer, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

def classification_report_with_accuracy_score(y_true, y_pred):

    return precision_score(y_true, y_pred,average="weighted")

    
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
precision_score = cross_val_score(nb, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(precision_score) 
print(precision_score.mean())
print(precision_score.std())

[0.74179495 0.65628254 0.63657089 0.69385572 0.67765644]
0.6812321081408406
0.035936345081543364


### Macro Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    #print(classification_report(y_true, y_pred)) # print classification report
    return recall_score(y_true, y_pred,average="macro")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(nb, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score) 
print("Mean of recall",recall_score.mean())
print("STD of recall",recall_score.std())


[0.60071462 0.55406398 0.54216989 0.56825439 0.57425856]
Mean of recall 0.5678922887230518
STD of recall 0.019853614108451923


### Weighted Recall

In [ ]:
from sklearn.metrics import recall_score
def classification_report_with_accuracy_score(y_true, y_pred):

    #print(classification_report(y_true, y_pred)) # print classification report
    return recall_score(y_true, y_pred,average="weighted")





kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
# Nested CV with parameter optimization
recall_score = cross_val_score(nb, X=x, y=y, cv=kfold, 
               scoring=make_scorer(classification_report_with_accuracy_score))
print(recall_score) 
print("Mean of recall",recall_score.mean())
print("STD of recall",recall_score.std())


[0.47       0.43714286 0.42918455 0.43776824 0.46351931]
Mean of recall 0.4475229920294298
STD of recall 0.016126369970024


In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(nb, X=x, y=y, cv=kfold)
conf_mat = confusion_matrix(y, y_pred)
print(conf_mat)

[[ 959  125]
 [1807  606]]


## Accuracy

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold


kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
models = [rf_tfidf,rf_bow,mnb_tfidf,mnb_bow,dt_tfidf,dt_bow,lr_tfidf,lr_bow]
for model in models:
  acc = cross_val_score(model, X=cross_val_df['text'].values, y=cross_val_df['labels'].values, cv=kfold, 
               scoring="accuracy")
  print(f"THE {model} has {acc.mean()} accuracy and {acc.std()} standard deviation")

THE Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier())]) has 0.7935328019619865 accuracy and 0.01787655620375487 standard deviation
THE Pipeline(steps=[('vect', CountVectorizer()), ('clf', RandomForestClassifier())]) has 0.7903932147966483 accuracy and 0.01637100759351669 standard deviation
THE Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())]) has 0.7778099325567137 accuracy and 0.008967228714495155 standard deviation
THE Pipeline(steps=[('vect', CountVectorizer()), ('clf', MultinomialNB())]) has 0.7875315757204169 accuracy and 0.01481635654492127 standard deviation
THE Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', DecisionTreeClassifier())]) has 0.7454931534845697 accuracy and 0.012119082073449956 standard deviation
THE Pipeline(steps=[('vect', CountVectorizer()), ('clf', DecisionTreeClassi

In [ ]:
ft_models = [lr,dt,rf,nb,svc]
for model in ft_models:
  acc = cross_val_score(model, X=x, y=y, cv=kfold, 
               scoring="accuracy")
  print(f"THE {model} has {acc.mean()} accuracy and {acc.std()} standard deviation")

THE LogisticRegression() has 0.6891624770079706 accuracy and 0.0020793927715807253 standard deviation
THE DecisionTreeClassifier() has 0.6680008174943797 accuracy and 0.008704575434339591 standard deviation
THE RandomForestClassifier() has 0.7849519722051911 accuracy and 0.012327720106470942 standard deviation
THE GaussianNB() has 0.4475229920294298 accuracy and 0.016126369970024 standard deviation
THE LinearSVC() has 0.7483543838136113 accuracy and 0.005177970681698347 standard deviation


###